In [1]:
import sys
sys.path.append('/home/chujunyi/2_Program/1_code/1_construct_dataset')
from process_scrapy_data__generate_dti import *

In [2]:
dataset_name = 'GPCR' ###
pwd = '/home/chujunyi/2_Program/2_output_file/1_construct_dataset/2_obtain_new_data/' ###
filename = '2_GPCR_KEGGBRITE-Uniprot-DrugBank_DTIs.npz' ###
dti_data, dtis_keggid, dtis_dbid, drugs_keggid, drugs_dbid, targets_all = load_dti_data(pwd, filename)

dtis_keggid.shape = (5515, 2), dtis_dbid.shape = (446, 2), dti_data.shape = (5961, 2)
# Target = 166; # Drug = 1901: KEGG = 1764, DB = 137


# 爬取smiles, aa sequence

In [3]:
error_target = kegg_retrieve_target(targets_all, dataset_name)

100%|██████████| 166/166 [01:54<00:00,  1.30it/s]


In [4]:
error_kegg_drug = kegg_retrieve_drug(drugs_keggid, dataset_name)
error_kegg_drug_2 = kegg_retrieve_drug(error_kegg_drug, dataset_name)

100%|██████████| 110/110 [01:39<00:00,  1.12it/s]


In [5]:
error_db_drug = drugbank_retrieve_drug(drugs_dbid, dataset = dataset_name)
error_db_drug_2 = drugbank_retrieve_drug(error_db_drug, dataset = dataset_name)

100%|██████████| 4/4 [00:03<00:00,  1.13it/s]


In [6]:
len(error_kegg_drug_2), len(error_db_drug_2), error_kegg_drug_2[0],error_db_drug_2[0]

(109, 4, 'D08571', 'DB11130')

In [7]:
error_kegg_drug_3 = scrapy_kegg_nosmiles_id(error_kegg_drug_2, dataset_name)
len(error_kegg_drug_3)

100%|██████████| 109/109 [02:04<00:00,  1.20s/it]


92

In [8]:
error_kegg_drug_3.extend(error_db_drug_2)
len(error_kegg_drug_3)

96

# 删掉没找到smiles的dti

In [10]:
non_smiles_drugs_id_list = deepcopy(error_kegg_drug_3)
dti_data, drug_list, target_list = delete_no_smiles_dtis(dti_data, non_smiles_drugs_id_list)
drug_dict, drug_id_smiles = obtain_drug_id_smiles_df(drug_list, data_set = dataset_name, save_ = True)
assert drug_id_smiles.shape[0] == len(drug_dict) == len(drug_list)

  3%|▎         | 46/1805 [00:00<00:03, 459.28it/s]

Original data: dti_data.shape = (5961, 2)
After delete: dti_data.shape = (5785, 2), # drugs = 1805, # targets = 156


100%|██████████| 1805/1805 [00:03<00:00, 561.77it/s]


In [17]:
repeat_smi_id, all_repeat_id  = find_repeated_smi_id(drug_id_smiles)
all_replaced_dtis = obtain_replaced_repeated_dtis(dtis_dbid, dtis_keggid, repeat_smi_id, print_ = False)
final_dti_data_with_smiles = delete_impute_repeated_dtis(dti_data, all_repeat_id, all_replaced_dtis)

# repeat_smi = 116, # repeat_id = 241
# all_replaced_dtis =  527
Original dti_data.shape =  (5785, 2)
After delete, dti_data.shape =  (4856, 2)
After impute, final dti_data with smiles shape =  (5383, 3)


In [18]:
final_dti_data_with_smiles.shape, len(set(final_dti_data_with_smiles['smiles'])), len(set(final_dti_data_with_smiles['hsa_id']))

((5383, 3), 1680, 156)

In [19]:
pwd = '/home/chujunyi/2_Program/2_output_file/1_construct_dataset/3_drop_repeat_smiles_from2folder/'
filename = 'v2' + dataset_name + '_updated_drug_smiles_ids_drop_repeated.csv'
final_dti_data_with_smiles.to_csv(pwd + filename, index = False)

# Target

In [20]:
generate_all_target_fasta_file(targets_all, data_set = 'GPCR')